# <center> <font color='black'> <b>Hiểu về nhân tiếp tuyến thần kinh (Neural Tangent Kernel)

<center> <img src="images\ellipses107.gif" width="500">

### <font color='black'> <b>Ảnh gif này mô tả quá trình huấn luyện của một mạng nơ-ron. Tìm hiểu cách thực hiện nó bằng cách đọc bài viết dưới đây.

### <font color='black'> Một loạt các bài báo gần đây về lý thuyết học sâu (Deep Learning) đề cập đến chủ đề chung là phân tích mạng nơ-ron trong giới hạn <b>chiều rộng vô hạn (infinite-width)</b>. Lúc đầu, người ta thấy giới hạn này có vẻ không thực tế và thậm chí là vô nghĩa để nghiên cứu. Tuy nhiên, hóa ra các mạng nơ-ron trong chế độ này sẽ đơn giản hóa thành các <b>mô hình tuyến tính (linear models)</b> với một hạt nhân được gọi là <b>hạt nhân tiếp tuyến thần kinh (neural tangent kernel)</b>. Kỹ thuật gradient descent do đó sẽ rất đơn giản để nghiên cứu trong mạng này. Mặc dù điều này thoạt đầu nghe có vẻ hứa hẹn, nhưng kết quả thực nghiệm cho thấy rằng các mạng nơ-ron trong chế độ này hoạt động kém hơn so với các mạng được tham số hóa quá mức (over-parameterized) trong thực tế. Tuy nhiên, điều này vẫn cung cấp cái nhìn sâu sắc về mặt lý thuyết về một số khía cạnh nào đó của quá trình huấn luyện mạng nơ-ron, vì vậy nó rất đáng để nghiên cứu.

### Ngoài ra, chế độ hạt nhân này có thể xảy ra theo một tiêu chí chung hơn phụ thuộc vào <b>quy mô (scale)</b> của mô hình và không yêu cầu chiều rộng vô hạn cho mỗi lần huấn luyện. Trong bài viết này, người ta sẽ trình bày một phần giới thiệu đơn giản và trực quan về lý thuyết dẫn đến một <b>bằng chứng về sự hội tụ của gradient descent</b> dần về sai số huấn luyện bằng 0. Ta sẽ sử dụng một ví dụ 1-d đơn giản để từ đó dễ hình dung và giúp làm cho các ý tưởng dễ hiểu hơn.

# <b> <font color='black'> Cài đặt
---

### <font color='black'> Hãy bắt đầu với một ví dụ cực kỳ đơn giản và khái quát. Hãy làm việc với mạng đầu vào 1-D và đầu ra 1-D. Một mạng relu 2 lớp ẩn đơn giản với chiều rộng <i>m</i> sẽ được sử dụng.

<center> <img src="images\simple_nn.png" width="600">

### <b> Mô hình 1-D đơn giản mà chúng ta sẽ sử dụng cho các thí nghiệm ban đầu.

### <font color='black'> Chúng ta có thể vẽ sơ đồ biểu diễn kết quả khi hàm mạng khởi tạo các giá trị ngẫu nhiên:

<center> <img src="images\net_funcs.png" width="500">

### <b>100 mạng relu 2 lớp ẩn với độ rộng 100 được khởi tạo ngẫu nhiên.

### <font color='black'> Hóa ra các hàm này tương ứng với các mẫu được vẽ từ quy trình Gaussian khi ta đưa chiều rộng lớp ẩn đến vô hạn, nhưng vấn đề đó tốt nhất nên để lại cho một bài khác.

### Ta có một số ký hiệu như sau:
- ### Gọi $f(x, \pmb{w})$ là hàm mạng nơ-ron trong đó $x$ là đầu vào và $\pmb{w}$ là vectơ kết hợp của các trọng số <b>(đang nói về kích thước $p$)</b>.
- ### Trong ví dụ 1-D này, tập dữ liệu (dataset) của chúng ta sẽ chỉ là các điểm $(x, y)$. Giả sử có $N$ điểm trên, thì tập dữ liệu của chúng ta sẽ là: $\{\bar x_i, \bar y_i\}_{i=1}^{N}$.

### Đối với việc tìm hiểu mạng này, chúng ta sẽ thực hiện lại một cách tiếp cận đơn giản: chỉ cần <b> thực hiện full-batch gradient descent với hàm mất mát bình phương tối thiểu (least squares loss)</b>. Bây giờ, có lẽ chúng ta đã quen với việc viết hàm mất mát này như sau:

## $$L(\boldsymbol w) = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{2}(f(\bar x_i, \boldsymbol w) - \bar y_i)^2$$

### Nhưng chúng ta có thể đơn giản hóa công thức này bằng cách sử dụng một số ký hiệu vector.

- ### Đầu tiên, xếp tất cả các giá trị đầu ra của tập dữ liệu $\bar y_i$ vào một vectơ duy nhất có kích thước $N$ và gọi nó là $\pmb{\bar{y}}$.

- ### Tương tự, xếp tất cả các đầu ra của mô hình ứng với mỗi đầu vào, tức là $f(\bar x_i, \pmb{w})$ thành một vectơ dự đoán duy nhất $\pmb{y(w)}$ ∈ $\mathbb{R}^N$. Về cơ bản chúng ta có $\pmb{y(w)}_i = f(\bar x_i, \pmb{w})$. Điều này tương tự như việc xem xét hàm mạng nơ-ron $f(⋅, \pmb{w})$ như một vectơ duy nhất thuộc <b>không gian hàm (function space)</b>.

- ### Do đó, hàm mất mát của chúng ta có thể đơn giản hóa thành:
## $$L(\boldsymbol w) = \frac{1}{N} \cdot \frac{1}{2} \Vert \boldsymbol y(\boldsymbol w) - \boldsymbol{\bar y} \Vert^2_2$$

### Bây giờ, chúng ta sẽ không thay đổi kích thước $N$ của tập dữ liệu ở bất kỳ đâu, và do đó nó là một hằng số không cần thiết trong biểu thức này của hàm loss. Vì vậy, chúng ta có thể bỏ qua nó mà không ảnh hưởng đến bất kỳ kết quả nào khác, đồng thời làm cho công thức trông bớt lộn xộn hơn (chúng ta vẫn giữ $\frac{1}{2}$ vì nó sẽ làm cho các đạo hàm đẹp hơn).

## $$L(\boldsymbol w) =  \frac{1}{2} \Vert \boldsymbol y(\boldsymbol w) - \boldsymbol{\bar y} \Vert^2_2$$

## <font color='black'>
<center> <img src="images\notation.png" width="700">

### <b>Hình trên giải thích ký hiệu vectơ. Chúng ta đang vector hóa mọi thứ một cách hiệu quả trên toàn bộ tập dữ liệu, trong trường hợp này để đơn giản ta cho kích thước là 2 (hai chấm màu xanh lam).
</center>

### Việc huấn luyện mô hình bây giờ chỉ là làm giảm thiểu giá trị hàm mất mát này bằng cách sử dụng gradient descent.
<center> <img src="images\training_nets_100.gif" width="500">

### <b>Huấn luyện 100 mạng ReLU sử dụng gradient descent trên hàm mất mát bình phương (square loss).
</center>

### Nếu chúng ta tạo một ảnh gif biểu diễn trọng số giữa hai lớp ẩn (ma trận $m \times m$) khi quá trình huấn luyện đang diễn ra, chúng ta sẽ quan sát thấy một điều rất thú vị:

<center><img src="images\wim022_width10.gif" width="250"'>
<img src="images\wim024_width100.gif" width="250">
<img src="images\wim025_width1000.gif" width="270">

### <b>Hình ảnh động</b> của các ma trận trọng số trong suốt quá trình huấn luyện. Chúng ta chỉ có thể thấy một số thay đổi đối với chiều rộng bằng 10, nhưng với chiều rộng lớn hơn thì trông hoàn toàn tĩnh (gần như không thay đổi).
</center>

### Đúng, đây thực sự là những ảnh gif động, nhưng <b>các trọng số gần như không thay đổi</b> đối với các mạng có chiều rộng lớn hơn! Có vẻ như những mô hình này khá <i>"lười biếng"</i>! Chúng ta có thể thấy điều này một cách định lượng, bằng cách xem xét <i>sự thay đổi tương đối (relative change)</i> trong quy chuẩn của vectơ trọng số từ khi khởi tạo:
## $$\frac{\Vert \boldsymbol w(n) - \boldsymbol w_0 \Vert_2}{\Vert \boldsymbol w_0 \Vert_2}$$

### Nếu các trọng số tăng gấp đôi, sự thay đổi tương đối này sẽ là <b>2</b> bất kể kích thước của các lớp ẩn. Bây giờ nếu chúng ta vẽ biểu đồ cho các mạng mà chúng ta đã huấn luyện ở trên:

<center><img src="images\losses_3widths.png" width="500"'>
<img src="images\weightchange_3widths.png" width="500">

### Các đường cong biểu diễn hàm loss và mức độ thay đổi của các trọng số trong quá trình huấn luyện. Các chuẩn trọng số được tính toán bằng cách sử dụng TẤT CẢ các tham số trong mô hình đã được xếp chồng lên nhau thành một vectơ duy nhất. Ngoài ra, tất cả các siêu tham số khác như tốc độ học (learning rate) được giữ cố định.
</center>

### Các trọng số không thay đổi nhiều đối với các chiều rộng lớn hơn của lớp ẩn. Vậy điều tốt nhất ta nên làm là gì nếu một số biến số không thay đổi nhiều? <i>Gợi ý: Xem khóa học Giải tích Calculus 101</i>

# <b> <font color='black'> Chỉ cần dùng Taylor mở rộng
---

### <font color=black> Đúng vậy, chúng ta chỉ cần sử dụng chuỗi Taylor mở rộng trong hàm mạng <b>với sự quan tâm đến trọng số</b> xung quanh <b>việc khởi tạo</b> nó.

## $$f(x, {\color{red}\boldsymbol w}) \approx f(x, \boldsymbol{w_0}) + \nabla_{\boldsymbol w}f(x, \boldsymbol{w_0})^T ({\color{red} \boldsymbol w} - \boldsymbol{w_0})$$

### Chúng ta đã biến hàm mạng phi tuyến tính phức tạp trở thành một hàm <b>tuyến tính</b> đơn giản của các trọng số. Nếu sử dụng ký hiệu vectơ ngắn gọn hơn cho các kết quả đầu ra của mô hình trên một tập dữ liệu cụ thể, chúng ta có thể viết lại biểu thức trên thành:

## $$\boldsymbol y({\color{red}\boldsymbol w}) \approx y(\boldsymbol{w_0}) + \nabla_{\boldsymbol w} \boldsymbol y(\boldsymbol{w_0})^T({\color{red}\boldsymbol w} - \boldsymbol{w_0})$$

### Với các kích thước của ma trận được thể hiện rõ ràng như sau:
<center><img src="images\taylor.png" width="1000">

### Hình ảnh này làm rõ kích thước của tất cả các vectơ và ma trận.
</center>
<br>

### Rất nhiều thứ ở đây như đầu ra ban đầu $y(\boldsymbol{w_0})$ và mô hình Jacobian $\nabla_{\boldsymbol w} \boldsymbol y(\boldsymbol{w_0})$ chỉ là những hằng số. Nếu để ý vào sự phụ thuộc của biểu thức vào ${\color{red}\boldsymbol w}$ - ta sẽ thấy xấp xỉ trên là một <b>mô hình tuyến tính theo các trọng số</b>, vì vậy việc làm giảm thiểu tổn thất bình phương nhỏ nhất (least squares loss) bây giờ chỉ là việc thực hiện <b>hồi quy tuyến tính (linear regression)!</b> Tuy nhiên, lưu ý rằng mô hình vẫn là hàm <b>phi tuyến tính theo đầu vào</b>, bởi vì việc tìm gradient của mô hình chắc chắn không phải là một phép toán tuyến tính. Trên thực tế, đây chỉ là một mô hình tuyến tính sử dụng **feature map** $\boldsymbol \phi(x)$ là vectơ gradient tại lúc khởi tạo:

## $$\boldsymbol \phi({\color{blue}x}) = \nabla_{\boldsymbol w} f({\color{blue}x}, \boldsymbol{w_0})$$

### Feature map này một cách tự nhiên <i>tạo ra một hạt nhân (kernel)</i> tại đầu vào, được gọi là <b>hạt nhân tiếp tuyến thần kinh (neural tangent kernel)</b>. Chúng ta sẽ đi sâu vào các chi tiết của hạt nhân này sau khi chúng ta xem xét về gradient flows, nhưng trước hết, chúng ta hãy cố gắng chứng minh về mặt lý thuyết phép xấp xỉ tuyến tính này.

# <b> <font color='black'> Khi nào thì xấp xỉ chính xác?
---

### <font color=black> Mô hình được tuyến tính hóa rất tốt để phân tích, nhưng chỉ khi nó thực sự là một mô hình xấp xỉ chính xác của mô hình phi tuyến tính. Chúng ta hãy cố gắng tìm ra một tiêu chí định lượng cho thời điểm khi nào mà sự xấp xỉ gần đúng này hoạt động tốt. Điều này được phỏng theo bài báo của Chizat và Bach [On Lazy Training in Differentiable Programming](https://arxiv.org/abs/1812.07956) (trong đó <b>chế độ lười biếng (lazy regime)</b> chỉ là một tên gọi khác của phương pháp xấp xỉ tuyến tính này).

### Trước hết, chúng ta chỉ quan tâm đến việc mô hình là tuyến tính khi chúng ta tối ưu hóa nó bằng cách sử dụng gradient descent:

## $$\boldsymbol w_1 = \boldsymbol w_0 - \eta  \nabla_{\boldsymbol w} L(\boldsymbol w_0)$$

### Với tốc độ học $\eta$ đủ nhỏ, chúng ta biết rằng giá trị hàm loss luôn giảm xuống khi chúng ta chạy gradient descent. Vì vậy, kết quả đầu ra của mô hình $\boldsymbol y(\boldsymbol{w_n})$ luôn tiến gần đến các giá trị nhãn thực sự (true labels) $\boldsymbol{\bar y}$. Sử dụng điều này, chúng ta có thể ràng buộc mức thay đổi ròng (net change) như sau:

## $$\text{net change in }\boldsymbol y (\boldsymbol w) \lesssim \Vert \boldsymbol y(\boldsymbol{w_0}) - \boldsymbol{\bar y} \Vert$$

### Giờ đây, chúng ta có thể định lượng "khoảng cách" mà chúng ta di chuyển trong không gian tham số với một phép xấp xỉ gần đúng bậc nhất,

## $$\text{distance } d \text{ moved in $w$ space} \approx \frac{\text{net change in }\boldsymbol y (\boldsymbol w)}{\text{rate of change of } \boldsymbol y \text{ w.r.t }\boldsymbol w} = \frac{\Vert \boldsymbol y(\boldsymbol{w_0}) - \boldsymbol{\bar y} \Vert}{\Vert \nabla_w \boldsymbol y(\boldsymbol{w_0})\Vert}$$

### Phép chuẩn hóa Jacobian $\Vert \nabla_w \boldsymbol y(\boldsymbol{w_0})\Vert$ là một chuẩn toán tử. Nếu không quen với các phép chuẩn hóa, chúng ta có thể coi nó như một thước đo độ kéo dài tối đa mà ma trận có thể thực hiện trên một vectơ (hoặc thậm chí đơn giản hơn, chỉ cần coi nó như một thước đo 'độ lớn' của ma trận).

### <font color=black> Để có được sự thay đổi trong Jacobian, chúng ta có thể sử dụng khoảng cách *d* này cùng với **Hessian $\nabla_w^2 \boldsymbol y (\boldsymbol{w_0})$** của đầu ra mô hình liên quan đến trọng số. Phép tính Hessian trong trường hợp này là một tensor bậc 3 (vẫn là một toán tử tuyến tính) , và ta có thể coi nó giống như việc xếp chồng các ma trận Hessian cho mỗi đầu ra lại với nhau để có được một "khối" Hessian 3-d. Một lần nữa, ta chỉ cần coi norm như như là một số đo lường độ lớn của Hessian, hoặc *tốc độ thay đổi (rate of change)* của Jacobian.

## $$\text{change in model Jacobian} \approx \text{distance } d \times \text{rate of change of the Jacobian} = d \cdot \Vert \nabla_w^2 \boldsymbol y (\boldsymbol{w_0})\Vert$$

### Điều chúng ta thực sự quan tâm khi muốn mô hình tuyến tính không chỉ là sự thay đổi trong Jacobian của nó, mà là **sự thay đổi tương đối (relative change)**. Cụ thể, chúng ta muốn thay đổi tương đối này càng nhỏ càng tốt ($\ll 1$):

## $$\text{relative change in model Jacobian} \approx  \frac{d \cdot \text{rate of change of Jacobian}}{\text{norm of Jacobian}} = \frac{d \cdot \Vert \nabla_w^2 \boldsymbol y (\boldsymbol{w_0})\Vert}{\Vert \nabla_w \boldsymbol y(\boldsymbol{w_0})\Vert} = {\color{blue} \Vert( \boldsymbol y (\boldsymbol w_0) - \boldsymbol{\bar y})\Vert \frac{\Vert \nabla_w^2 \boldsymbol {y(w_0)} \Vert}{\Vert \nabla_w \boldsymbol {y(w_0)} \Vert^2}}{\color{red} \ll 1}$$

### Hãy gọi đại lượng màu xanh lam là $\color{blue} \kappa(\boldsymbol{w_0})$. Điều kiện $\color{blue} \kappa(\boldsymbol{w_0}) \color{red} \ll 1$ có thể được **tóm tắt một cách trực quan** như sau:

### *Lượng thay đổi trong $\boldsymbol w$ yêu cầu tạo ra một sự thay đổi $\Vert( \boldsymbol y (\boldsymbol w_0) - \boldsymbol{\bar y})\Vert$ theo $\boldsymbol y$ để gây ra sự thay đổi không đáng kể trong Jacobian $\nabla_w \boldsymbol {y(w)}$.*

### Điều này có nghĩa là mô hình rất **gần với xấp xỉ tuyến tính của nó**.

### <font color=b> Bây giờ chúng ta cần hiểu $\color{blue} \kappa(\boldsymbol{w_0})$ thay đổi như thế nào với chiều rộng lớp ẩn $m$ của mạng nơ-ron của chúng ta. Nó chỉ ra rằng $\color{blue} \kappa \to 0$ khi chiều rộng $m \to \infty$. Chúng ta cần phải cẩn thận một chút ở đây, bởi vì $\kappa(\boldsymbol{w_0})$ là một biến ngẫu nhiên vì bản thân sự khởi tạo của $\boldsymbol{w_0}$ là ngẫu nhiên. Khi nói $\kappa \to 0$ có ý nghĩa là *kỳ vọng* của nó sẽ bằng 0. Điều này chỉ đúng nếu các trọng số được **khởi tạo đúng cách**. Cụ thể, chúng cần phải là các biến ngẫu nhiên Gaussian có trung bình bằng không - độc lập - với phương sai tỷ lệ nghịch với kích thước của lớp đầu vào. (Đây được gọi là *khởi tạo LeCun*, sẽ nói thêm về điều này trong phần tiếp theo). Kết quả này giải thích tại sao các trọng số thay đổi ít hơn trong khi huấn luyện cho các mạng nơ-ron có chiều rộng lớn hơn.

### Hiểu được lý do tại sao kết quả này đúng với trường hợp tổng quát là khá phức tạp, vì vậy ta sẽ trình bày một dẫn xuất cho trường hợp đơn giản hơn là chỉ có **một** lớp ẩn.

### Một lời giải thích trực quan cho lý do tại sao điều này xảy ra như sau: chiều rộng lớn có nghĩa là có nhiều nơ-ron hơn ảnh hưởng đến đầu ra. Một thay đổi nhỏ trong tất cả các trọng số nơ-ron này có thể dẫn đến sự thay đổi rất lớn trong đầu ra, do đó, các nơ-ron cần phải di chuyển rất ít để phù hợp với dữ liệu. Nếu các trọng số di chuyển ít hơn, thì phép gần đúng tuyến tính sẽ chính xác hơn. Khi chiều rộng tăng lên, số lượng nơ-ron này giảm đi, do đó, mô hình tiến gần hơn đến xấp xỉ tuyến tính của nó. Nếu bạn hài lòng với việc chỉ tin vào trực giác này, bạn có thể bỏ qua phần chứng minh và chuyển sang phần tiếp theo.

# <b> <font color='black'> Chứng minh trên mạng một lớp ẩn (One Hidden Layer Network)
---

### Hãy xem xét một mạng nơ-ron một lớp ẩn, một lần nữa với các đầu vào và đầu ra 1 chiều và không có các hệ số bias để mạng của chúng ta đơn giản hơn. Hàm mạng chỉ đơn giản là:

## $$f(x, \boldsymbol w) = \sum_{i=1}^m b_i \sigma (a_i x)$$

### Trong đó:

- ### $\sigma$ là một hàm kích hoạt liên tục *(smooth twice-differentiable)*. (Lưu ý: ReLU không phải *twice-differentiable*, nhưng chúng ta có thể xấp xỉ nó với một phiên bản liên tục và chúng ta có thể dùng nó).
- ### Các trọng số giữa các lớp $a_i$ và $b_i$ được xếp chồng lên nhau thành một vectơ tham số $w \in \mathbb{R}^{2m}$.

### Chúng ta khởi tạo các trọng số bằng cách sử dụng *khởi tạo LeCun*, về cơ bản nó làm cho **tất cả các kích hoạt có phương sai không đổi (all the activations have constant variance)** bất kể chiều rộng của lớp trước đó. Trong trường hợp này, có nghĩa là:
- ### $a_i \sim \mathcal{N}(0, 1)$
- ### $b_i \sim \mathcal{N}(0, {\color{red}\frac{1}{m}})$ (thêm $m$ trong này để cho phương sai đơn vị)

<center>
<img src=images/one_layer_nn.png width=500>

### <b>Chúng ta sẽ chứng minh rằng mạng được đơn giản hóa này gần với sự tuyến tính hóa của nó khi chiều rộng tăng lên đến vô cùng.
</center>

### <font color=b> Để phân tích các tiệm cận theo chiều rộng của $\color{blue} \kappa(\boldsymbol{w_0})$, chúng ta cần tìm mô hình Jacobian và Hessians, là một ma trận và một tensor bậc 3. Bám sát chủ đề đơn giản hóa, chúng ta sẽ giới hạn phân tích của mình trong **tập dữ liệu chỉ một điểm** $(x, \bar y)$, để Jacobian trở thành vectơ gradient đơn giản và Hessian trở thành ma trận. Do đó, đầu ra của chúng ta **chỉ là một đại lượng vô hướng**: $y(\boldsymbol{w_0}) = f(x, \boldsymbol{w_0})$.

### Được rồi, bây giờ chúng ta đã giải quyết xong tất cả việc đơn giản hóa, bây giờ hãy giải quyết từng phần một với $\color{blue} \kappa(\boldsymbol{w_0})$. Hãy nhớ rằng chúng ta chỉ quan tâm đến các tiệm cận của $\color{blue} \kappa(\boldsymbol{w_0})$ khi chiều rộng $m \to \infty$.

### Trước hết, do quá trình khởi tạo cụ thể, đầu ra mô hình ít nhiều không đổi với chiều rộng, vì vậy $\Vert y(\boldsymbol {w_0}) - \bar y \Vert\sim \mathcal{O}(1)$ (giải thích ký hiệu big-O là hành vi tiệm cận *kỳ vọng* ​​khi chiều rộng $m \to \infty$). Điều này có nghĩa là chúng ta có thể bỏ qua thuật ngữ này và chúng ta chỉ còn tỷ lệ sau:
## $$\kappa(w_0) \sim \frac{\text{norm of Hessian}}{\text{(norm of gradient)}^2}$$

### Hãy giải thích hai thuật ngữ trên.


## <b><font color='black'> Đạo hàm (Gradient)

### Việc tìm ra gradient chỉ là vấn đề của việc áp dụng quy tắc chuỗi (chain rule) (về cơ bản nó là lan truyền ngược của hai lớp). Đầu ra là:
## $$y = \sum_{i=1}^m b_i \sigma (a_i x)$$
### Ta đã bỏ $\boldsymbol{w_0}$ vì tất cả các đạo hàm sẽ được đánh giá khi khởi tạo, vì vậy nó là không cần thiết.

### Chúng ta có thể viết ra các gradient cho hai lớp:

## $$\frac{\partial y}{\partial a_i} = {\color{red}b_i} \cdot \sigma'(a_i x) x \text{ and }\frac{\partial y}{\partial b_i} = \sigma(a_i x)$$

### Trong đó $\sigma'$ là đạo hàm của hàm kích hoạt. Chuẩn gradient là:

## $$\Vert \nabla y \Vert = \sqrt{\left( \frac{\partial y}{\partial a_1} \right)^2 + \dots + \left( \frac{\partial y}{\partial a_m} \right)^2 + \left( \frac{\partial y}{\partial b_1} \right)^2 + \dots + \left( \frac{\partial y}{\partial b_m} \right)^2}$$

### Lưu ý rằng:
- ### Chúng ta giả định rằng $x$ chỉ là một **hằng số**.
- ### $\sigma(a_i x)$ và $\sigma'(a_i x)$ là các biến ngẫu nhiên có **phương sai không đổi** khi chiều rộng tăng. (Ta cần $\sigma$ trở thành Lipschitz ở đây) (Xem thêm về Lipschitz trong toán Tối ưu hóa).
- ### **Chỉ có phương sai của $\color{red} b_i$ thay đổi theo chiều rộng**. Cụ thể, nó có tên là $\mathcal{O}(\frac{1}{m})$ vì ta dùng khởi tạo LeCun.

### <font color=b> Khi chúng ta bình phương các biến ngẫu nhiên này, phương sai chuyển thành giá trị trung bình. Vì vậy, bên trong căn bậc hai, chúng ta sẽ thêm:

- ### Các biến $m$ với **trung bình** $\mathcal{O}(\frac{1}{m})$ (Biểu thức $\big( \frac{\partial y}{\partial a_i}\big)^2$). Chúng tiến đến 0 khi $m \to \infty$, vì vậy chúng ta có thể bỏ qua chúng.
- ### Các biến $m$ khác với **trung bình** $\mathcal{O}(\frac{1}{m})$ (Biểu thức $\big( \frac{\partial y}{\partial b_i}\big)^2$).

### Lưu ý rằng các số hạng trung bình không đổi, $\big( \frac{\partial y}{\partial b_i}\big)^2 = \sigma^2 (a_i x)$ thực sự là các biến ngẫu nhiên iid, vì $a_i$ là iid *(iid có nghĩa là "Độc lập và phân phối giống hệt nhau")* (hãy nhớ rằng $x$ không phải là ngẫu nhiên, nhưng chỉ là một giá trị cố định). Chúng ta có thể rút ra một thừa số $\sqrt m$ từ dưới căn bậc hai và nhận được biểu thức sau:
## $$\Vert \nabla y \Vert \sim \sqrt{m} \sqrt{\frac{1}{m} (\sigma^2 (a_1 x) + \dots + \sigma^2(a_m x))}$$

### Biểu thức trong căn bậc hai là một trung bình thực nghiệm đẹp của các biến ngẫu nhiên iid. Theo luật số lớn, nó tiến tới giá trị trung bình thực khi $m \to \infty$. Mức trung bình thực sự này là một hằng số khi chúng ta chia tỷ lệ $m$, và quan trọng hơn là có **tính xác định**. Vì vậy, chuẩn gradient chính nó tiếp cận một giá trị xác định là $\sqrt{m}$. Điều này có nghĩa rằng,

## $$\color{green} \Vert \nabla y \Vert \sim \mathcal{O}(\sqrt{m}) \text{ and } \Vert \nabla y \Vert \sim \Omega(\sqrt{m})$$

### Hãy nhớ lại big-$\Omega$ giống như big-O, nhưng là một tiệm cận dưới thay vì một giới hạn trên. Chúng ta cần một giới hạn dưới cho tiêu chuẩn gradient vì nó nằm ở mẫu số của biểu thức cho $\kappa$ (mà chúng ta đang cố gắng giới hạn trên).

### Chúng ta có thể xác minh theo kinh nghiệm hành vi tiệm cận này khá dễ dàng:
<center>
<img src=images/gradient.png width=500>

### Chuẩn gradient của một mạng tanh lớp ẩn đơn lúc khởi tạo khi chiều rộng tăng lên. Giá trị đầu vào $x$ là cố định. Như mong đợi, nó phát triển tương ứng $\sim \sqrt m$.
</center>

## <b><font color='black'> Hessian

### Chúng ta sắp hoàn thành, chỉ là một đạo hàm khác. Điều này thoạt đầu có vẻ hơi khó khăn, nhưng nó chỉ đơn giản là hơi khác biệt hơn một chút. Trước hết, vectơ gradient có hai khối ($\boldsymbol a$ và $\boldsymbol b$ là các vectơ trọng số cho lớp thứ nhất và lớp thứ hai):
## $$\nabla y = \begin{bmatrix}\frac{\partial y}{\partial \boldsymbol a} \\ \frac{\partial y}{\partial \boldsymbol b} \end{bmatrix}$$

### Do đó, Hessian sẽ có cấu trúc này:
## $$\nabla^2 y = \begin{bmatrix}\frac{\partial^2 y}{\partial \boldsymbol a^2} & \frac{\partial^2 y}{\partial \boldsymbol b \partial \boldsymbol a} \\\frac{\partial^2 y}{\partial \boldsymbol a \partial \boldsymbol b} & \frac{\partial^2 y}{\partial \boldsymbol b^2}\end{bmatrix}$$

### Chúng ta hãy phân tích từng khối một. Nhắc lại các gradient:
## $$\frac{\partial y}{\partial a_i} = {\color{red}b_i} \cdot \sigma'(a_i x) x \text{ and }\frac{\partial y}{\partial b_i} = \sigma(a_i x)$$

### Đầu tiên, các khối trên đường chéo:
## $$\begin{align*}\frac{\partial}{\partial a_j} \left( \frac{\partial y}{\partial a_i} \right) &= \frac{\partial}{\partial a_j} ({\color{red}b_i} \cdot \sigma'(a_i x) x) \\ &= {\color{red}b_i} \cdot \sigma''(a_i x) x^2 \text{  if  } i=j \text{ and 0 otherwise}\end{align*}$$

### Trong đó $\sigma''$ là đạo hàm bậc hai của hàm kích hoạt. Bây giờ, khối tiếp theo trên đường chéo là:
## $$\frac{\partial}{\partial b_j} \left( \frac{\partial y}{\partial b_i} \right) = \frac{\partial}{\partial b_j} (\sigma(a_i x)) = 0$$

### Vì vậy, khối đầu tiên trên đường chéo là một ma trận đường chéo đẹp và khối thứ hai hoàn toàn bằng 0. Hai khối ngoài đường chéo phải bằng nhau vì Hessian là đối xứng, vì vậy chúng ta chỉ cần tìm một trong số chúng.
## $$\begin{align*}\frac{\partial}{\partial a_j} \left( \frac{\partial y}{\partial b_i} \right) &= \frac{\partial}{\partial a_j} (\sigma(a_i x)) \\&= \sigma'(a_i x) x \text{  if  } i=j \text{ and 0 otherwise}\end{align*}$$

### <font color=b> Một lần nữa, các khối này cũng là các ma trận đường chéo đẹp. Bây giờ, thuật ngữ duy nhất với $\color{red} b_i$ là khối đường chéo đầu tiên. Chỉ những mục này có phương sai là $\mathcal{O}(\frac{1}{m})$, trong khi hai khối nằm ngoài đường chéo có các mục với phương sai không đổi. Hình ảnh này sẽ trình bày rõ ràng:
<center>
<img src=images/hessian_mat.png width=500>

### <b>Cấu trúc của Hessian trong mạng của chúng ta. Ba khối là ma trận đường chéo đẹp.
</center>

### Chúng ta cần tìm chuẩn toán tử của ma trận này. Chuẩn toán tử không thay đổi nếu chúng ta hoán đổi xung quanh các hàng (vì đó là một phép biến đổi trực giao), vì vậy thay vào đó chúng ta có thể tìm chuẩn của ma trận tam giác dưới đây:
<center>
<img src=images/hessian_mat2.png width=500>

### Hoán đổi các hàng $m$ ở trên với các hàng $m$ ở dưới của Hessian tương đương với việc nhân ở bên trái với một ma trận hoán vị trực giao $P$. Điều này không thay đổi maximum singular value, là operator norm. Cũng lưu ý rằng ma trận này tiến tới ma trận đường chéo khi $m \to \infty$.
</center>

### <font color=b> Bây giờ, các mục đường chéo là $\sigma'(a_i x) x$. Đây là các biến ngẫu nhiên **có giới hạn (bounded)** vì chúng ta đã giả định hàm kích hoạt là Lipschitz và đầu vào cũng được cố định (và có giới hạn). Khi chúng ta đưa chiều rộng đến vô cùng, các mục nhập trong khối ngoài đường chéo sẽ về 0, bởi vì phương sai của $\color{red} b_i$ tiến về 0 và giá trị trung bình của chúng cũng bằng 0. Điều này có nghĩa là Hessian hoán vị tiến tới ma trận đường chéo với các mục nhập bị giới hạn khi $m \to \infty$. Do đó, chuẩn toán tử của ma trận này (đơn giản là phần tử lớn nhất trên đường chéo trong trường hợp này) cũng là giới hạn tiệm cận! Nói cách khác,
## $$\color{purple} \Vert \nabla^2 y \Vert \sim \mathcal{O}(1)$$

### Thử nghiệm với mạng tanh đơn giản sẽ xác nhận điều này:
<center>
<img src=images/hessian.png width=500>

### Chuẩn toán tử của hessian của một mạng tanh lớp ẩn đơn khi khởi tạo khi chiều rộng tăng lên. Giá trị đầu vào $x$ là cố định. Như dự đoán, nó gần như không đổi.
</center>

## <b><font color='black'> Kết hợp chúng lại với nhau

### Bây giờ vấn đề chỉ là thay thế trong biểu thức của $\kappa$:
## $$\kappa(w_0) \sim \frac{\text{norm of Hessian}}{\text{(norm of gradient)}^2} \sim \frac{\color{purple}\mathcal{O}(1)}{\color{green}\Omega(\sqrt{m})^2} \sim \color{red}\mathcal{O}\left(\frac{1}{m}\right)$$

### Rõ ràng, điều này có nghĩa là $\kappa \to 0$ khi $m \to \infty$. Nói cách khác, xấp xỉ tuyến tính tốt hơn khi độ rộng lớn hơn! Chúng ta có thể xác nhận điều này cho mạng tanh:

<center>
<img src=images/kappa.png width=500>

### Sự giảm dần của $\kappa(\boldsymbol{w_0})$ theo chiều rộng. Giá trị đầu vào $x$ là cố định. Quả nhiên, nó rơi xuống như là $\mathcal{O}(\frac{1}{m})$. Điều này có nghĩa là mạng nơ-ron tiến gần đến độ tuyến tính hóa của nó khi chiều rộng tăng lên.
</center>

### <font color=b> Ta sẽ không trình bày tất cả các chi tiết của chứng minh này và tập trung vào các lập luận trực quan hơn.

### Một vài nhận xét:
- ### Khởi tạo LeCun là rất quan trọng để điều kiện này được duy trì. Nếu chúng ta chọn một phương sai khởi tạo nhỏ hơn, mạng sẽ không tiếp cận chế độ tuyến tính với độ rộng vô hạn.
- ### Đạo hàm này chỉ dành cho một giá trị của đầu vào $x$, nhưng bạn có thể dễ dàng mở rộng nó sang các kích thước tập dữ liệu lớn hơn bằng cách sử dụng một số kết quả đơn giản về tiêu chuẩn của ma trận khối.
- ### Ta đã bỏ qua các bias, nhưng chúng sẽ không ảnh hưởng đến kết quả cuối cùng và cũng khá dễ tính.
- ### Xử lý với các đầu vào và đầu ra có chiều lớn hơn cũng có thể thực hiện được trong cùng một mạch, nhưng việc tính toán đại số trở nên khá phức tạp.
- ### Mở rộng ra nhiều lớp không đáng kể, nhưng kết quả vẫn được duy trì. Nó đại khái liên quan đến việc quy nạp trên các lớp bằng cách gửi từng độ rộng của chúng đến vô cùng, và theo dõi các gradient bằng cách sử dụng các phương trình back-prop (lan truyền ngược). Nếu bạn đủ can đảm, bạn có thể đi sâu vào các chứng minh trong bài báo [Neural Tangent Kernel](https://arxiv.org/abs/1806.07572), hoặc là chứng minh trong bài báo [On Exact Computation in an Infinitely Wide Net](https://arxiv.org/abs/1904.11955) có vẻ dễ tiếp cận hơn một chút.

### Bạn có thể tìm code đã viết để tạo các đồ thị tiệm cận trong [sổ tay colab](https://colab.research.google.com/drive/1BsqUbINgaEHotlDcYZVVnWnDn_Bk86zk) đơn giản này.

---
### <font color=b> Mặc dù đây là một kết quả tiện lợi khi một mạng nơ-ron hoạt động tuyến tính trong các tham số của nó, hóa ra điều kiện **chung và đơn giản hơn** nhiều lại dễ dàng thu được.

# <b> <font color='black'> Mở rộng quy mô đầu ra
---

### Hãy nhìn lại biểu thức của $\color{blue} \kappa(\boldsymbol{w_0})$:
## $$\color{blue} \kappa(\boldsymbol{w_0}) = {\color{blue} \Vert( \boldsymbol y (\boldsymbol w_0) - \boldsymbol{\bar y})\Vert \frac{\Vert \nabla_w^2 \boldsymbol {y(w_0)} \Vert}{\Vert \nabla_w \boldsymbol {y(w_0)} \Vert^2}}$$

### Ta sẽ làm một việc rất đơn giản, đó là chỉ cần nhân đầu ra của mô hình của chúng ta với một số hệ số $\alpha$.
## $$\kappa_{\color{red}\alpha}(\boldsymbol{w_0}) = { \Vert( {\color{red}\alpha}\boldsymbol y (\boldsymbol {w_0}) - \boldsymbol{\bar y})\Vert \frac{\Vert \nabla_w^2 {\color{red}\alpha}\boldsymbol {y(w_0)} \Vert}{\Vert \nabla_w {\color{red}\alpha}\boldsymbol {y(w_0)} \Vert^2}}$$
### Đúng vậy, điều này thực sự chỉ **thay đổi lại tỷ lệ mô hình**. Biểu thức $\Vert( \boldsymbol y (\boldsymbol {w_0}) - \boldsymbol{\bar y})\Vert$ là rắc rối và chúng ta có thể loại bỏ nó bằng cách giả định rằng mô hình của chúng ta **luôn xuất ra 0 khi khởi tạo**, tức là $\boldsymbol y (\boldsymbol {w_0}) = 0$. (Chúng ta có thể làm cho bất kỳ mô hình nào có được điều này bằng cách trừ đi một bản sao của nó khi khởi tạo.)

## $$\implies \kappa_{\color{red}\alpha}(\boldsymbol{w_0}) \sim \frac{\Vert \nabla_w^2 {\color{red}\alpha} \boldsymbol {y(w_0)} \Vert}{\Vert \nabla_w {\color{red}\alpha} \boldsymbol {y(w_0)} \Vert^2} = \frac{ {\color{red}\alpha} \Vert \nabla_w^2 \boldsymbol {y(w_0)} \Vert}{ {\color{red}\alpha^2}\Vert \nabla_w \boldsymbol {y(w_0)} \Vert^2} = {\color{red}\frac{1}{\alpha}}{ \frac{\Vert \nabla_w^2 \boldsymbol {y(w_0)} \Vert}{\Vert \nabla_w \boldsymbol {y(w_0)} \Vert^2}}$$

### Chúng ta có thể làm cho mô hình tuyến tính (lấy $\color{blue}\kappa(\boldsymbol{w_0}) \to 0$) bằng cách đơn giản là cho $\color{red} \alpha \to \infty$! Theo đó, các tác giả của bài báo về huấn luyện lười biếng (lazy training) đã gọi đại lượng $\kappa(w_0)$ là tỷ lệ tương đối nghịch đảo *(inverse relative scale)* của mô hình.

### <font color=b> Một điều quan trọng cần lưu ý là điều này hoạt động đối với bất kỳ **mô hình phi tuyến tính nào** và không dành riêng cho mạng nơ-ron (mặc dù cần phải "twice differentiable"). Vì vậy, để hình dung điều này, ta sẽ tạo ra một mô hình mẫu 1-D "điên rồ". Mô hình này có *một trọng số* $w$ được khởi tạo là $w_0 = 0.4$.
## $$f(x, w) = (wx + w^2 x + \sin(e^{0.1w}))(w - 0.4)$$

### Nó cũng đáp ứng yêu cầu của chúng ta rằng đầu ra là 0 khi khởi tạo. Để xem ảnh hưởng của $\color{red} \alpha$ đến độ tuyến tính của mô hình, chúng ta chỉ cần nhìn vào giá trị của hàm tại một *giá trị cụ thể của $x$*, giả sử $x = 1.5$. Thay đổi $\color{red} \alpha$, chúng ta nhận được:

<center>
<img src=images/model_alpha.gif width=500>

### Thay đổi $\color{red} \alpha$ (hệ số tỷ lệ đầu ra) cho mô hình ví dụ 1 tham số của chúng ta được đánh giá tại một giá trị $x$ cụ thể. Qua quan sát ta thấy rằng sự tuyến tính hóa (về cơ bản là đường tiếp tuyến trong trường hợp này) tiếp cận với hàm thực tế khi tỷ lệ lớn hơn.
</center>

### <font color=b> Chúng ta cũng có thể xem xét bề mặt đồ thị loss cho một điểm dữ liệu duy nhất cho hai mô hình. Đồ thị loss tuyến tính sẽ là một parabol đẹp và ta hy vọng loss thực tế sẽ tiến gần hơn đến parabol này khi tăng giá trị $\color{red} \alpha$:

<center>
<img src=images/loss_alpha_fixed.gif width=500>
<img src=images/loss_alpha_dynamic.gif width=500>

### Các đồ thị loss đã chuẩn hóa. Quá trình chuẩn hóa được thực hiện đơn giản bằng cách chia cho $\alpha^2$ để loss thực tế không thay đổi theo $\alpha$. Ảnh gif bên trái và bên phải hiển thị cùng nội dung, nhưng ảnh bên phải cũng phóng to gần với $w=0.4$ để chúng ta có thể nhìn thấy sự khác biệt nhỏ rõ ràng hơn.
</center>

### <font color=b> Một vài nhận xét quan trọng:

- ### Loss tuyến tính (linearized loss) tiến gần hơn tới loss thực tế như chúng ta mong đợi.
- ### **Cực tiểu** của bề mặt hai loss cũng gần nhau hơn, và quan trọng hơn, chúng **cũng tiến gần đến giá trị khởi tạo**. Điều này hoàn toàn đúng với những quan sát trước đây của chúng ta rằng các trọng số hầu như không nhúc nhích khi chúng ta huấn luyện mô hình.

# <b> <font color='black'> Gradient Flow
---

### <font color=b> Chúng ta đã giải quyết xong câu hỏi khi nào mạng nơ-ron và các mô hình phi tuyến tính tổng quát hơn được xấp xỉ chính xác bởi các tuyến tính của chúng. Bây giờ chúng ta hãy tìm hiểu về động lực huấn luyện (training dynamics) của chúng trong gradient descent.
## $$\boldsymbol w_{k+1} = \boldsymbol w_k - \eta  \nabla_{\boldsymbol w} L(\boldsymbol w_k)$$
### Viết lại phương trình này một chút, chúng ta nhận được:
## $$\frac{\boldsymbol w_{k+1} - \boldsymbol w_{k}}{\eta} =  -\nabla_{\boldsymbol w} L(\boldsymbol w_k)$$

### Phần ở phía bên trái trông giống như một xấp xỉ *sai phân hữu hạn (finite-difference)* đối với một đạo hàm. Và phương trình này giống như một phương trình sai phân gần đúng với một *phương trình vi phân (differential equation)*. Nếu chúng ta coi tốc độ học là vô cùng nhỏ, ta có thể xem xét sự phát triển của các vectơ trọng số theo **thời gian** và viết ra phương trình vi phân này như sau:
## $$\frac{d \boldsymbol w(t)}{dt} = -\nabla_{\boldsymbol w} L(\boldsymbol w(t))$$
### Đây được gọi là *gradient flow*. Về bản chất, nó là một khoảng thời gian liên tục tương ứng với gradient descent tiêu chuẩn. Điểm mấu chốt chính là quỹ đạo của gradient descent trong không gian tham số gần đúng với quỹ đạo của nghiệm của phương trình vi phân này nếu tốc độ học đủ nhỏ. Để đơn giản hóa ký hiệu, ta sẽ biểu thị đạo hàm thời gian bằng dấu chấm: $\frac{d \boldsymbol w(t)}{dt} =  \boldsymbol {\dot w}(t)$. Và gradient flow được viết lại là:
## $$\boldsymbol{\dot w}(t) = - \nabla L(\boldsymbol{w}(t))$$

### Ngoài ra, hãy bỏ biến thời gian vì nó có thể được suy ra từ ngữ cảnh. Thay thế hàm mất mát và lấy gradient, chúng ta nhận được:
## $$\boldsymbol{\dot w} = - \nabla \boldsymbol{y}(\boldsymbol{w}) (\boldsymbol{y}(\boldsymbol{w}) - \boldsymbol{\bar y})$$

### <font color=b> Bây giờ chúng ta có thể suy ra động lực học của đầu ra mô hình $\boldsymbol{y}(\boldsymbol{w})$ (về cơ bản đây là **động lực học trong không gian hàm - dynamics in function space**) được tạo ra bởi gradient flow này bằng cách sử dụng quy tắc chuỗi (chain rule):
## $$\dot{\boldsymbol{y}}(\boldsymbol{w}) = \nabla \boldsymbol{y}(\boldsymbol{w})^T \boldsymbol{\dot w} = - {\color{red}\nabla \boldsymbol{y}(\boldsymbol{w})^T \nabla \boldsymbol{y}(\boldsymbol{w})} (\boldsymbol{y}(\boldsymbol{w}) - \boldsymbol{\bar y})$$
### Biểu thức được tô màu đỏ, $\color{red} \nabla \boldsymbol{y}(\boldsymbol{w})^T \nabla \boldsymbol{y}(\boldsymbol{w})$ được gọi là **hạt nhân tiếp tuyến thần kinh - neural tangent kernel** (viết tắt là NTK). Hãy đặt cho nó một ký hiệu $\color{red} \boldsymbol{H}(\boldsymbol{w})$.
### Nếu quay lại phần mà lần đầu tiên chúng ta dùng Taylor mở rộng hàm mạng, chúng ta thấy rằng mô hình tuyến tính hóa có ánh xạ đặc trưng (feature map) $\boldsymbol \phi({\color{blue}x}) = \nabla_{\boldsymbol w} f({\color{blue}x}, \boldsymbol{w_0})$ . Ma trận hạt nhân tương ứng với feature map này có được bằng cách lấy **pairwise inner products (một loại phép nhân ma trận)** giữa các feature map của tất cả các điểm dữ liệu. Đây chính xác là $\boldsymbol{H}(\boldsymbol{w_0})$!

<center>
<img src=images/ntk.png width=600>

### Nhân tiếp tuyến thần kinh khi khởi tạo bao gồm các phép nhân pair inner products giữa các feature map của các điểm dữ liệu. Lưu ý rằng điều này cũng có thể được hiểu là một phép nhân outer products trên các điểm dữ liệu (nhớ lại rằng $\bar x_i$ là các đầu vào trong tập dữ liệu của chúng ta).
</center>

### <font color=b> Nếu mô hình gần với xấp xỉ tuyến tính của nó, $\color{blue}\kappa(\boldsymbol{w_0}) \ll 1$, thì đầu ra Jacobian của mô hình **không thay đổi khi quá trình huấn luyện diễn ra**. Nói cách khác,
## $$\nabla \boldsymbol{y}(\boldsymbol{w}(t)) \approx \nabla \boldsymbol{y}(\boldsymbol{w}_0) \implies \boldsymbol{H}(\boldsymbol{w}(t)) \approx \boldsymbol{H}(\boldsymbol{w}_0)$$
### Đây được gọi là **chế độ hạt nhân (kernel regime)**, vì hạt nhân tiếp tuyến không đổi trong suốt quá trình huấn luyện. Động lực huấn luyện bây giờ giảm xuống một **phương trình vi phân tuyến tính thông thường (linear ordinary differential equation)** rất đơn giản:
## $$\dot{\boldsymbol{y}}(\boldsymbol{w}) = - \boldsymbol{H}(\boldsymbol{w_0}) (\boldsymbol{y}(\boldsymbol{w}) - \boldsymbol{\bar y})$$
### Rõ ràng, $\boldsymbol{y}(\boldsymbol{w}) = \boldsymbol{\bar y}$ là trạng thái cân bằng của ODE này, và nó tương ứng với loss huấn luyện bằng 0 - chính xác những gì chúng ta muốn. Chúng ta có thể thay đổi các biến trạng thái để loại bỏ $\boldsymbol{\bar y}$ bằng cách định nghĩa $\boldsymbol u = \boldsymbol{y}(\boldsymbol{w}) - \boldsymbol{\bar y}$. Quy trình đơn giản hóa thành:
## $$\boldsymbol{\dot u} = -\boldsymbol{H}(\boldsymbol{w_0}) \boldsymbol u$$

### <font color=b> Lời giải của ODE này được đưa ra bởi một ma trận theo cấp số nhân:
## $$\boldsymbol u(t) = \boldsymbol u(0)e^{-\boldsymbol{H}(\boldsymbol{w_0}) t}$$

### Vì mô hình của chúng ta được **over-parameterized** $p > n$, NTK $\nabla \boldsymbol{y}(\boldsymbol{w_0})^T \nabla \boldsymbol{y}(\boldsymbol{w_0})$ luôn *xác định dương (positive definite)* (bỏ qua bất kì sự suy giảm nào trong dataset có thể gây ra việc $\nabla \boldsymbol{y}(\boldsymbol{w_0})$ không có thứ hạng cột đầy đủ). Bằng cách thực hiện phân rã ma trận (spectral decomposition) trên NTK xác định dương, chúng ta có thể tách quỹ đạo của gradient flow thành các thành phần 1-d độc lập (các eigenvector) phân rã với tốc độ tỷ lệ với giá trị eigen tương ứng. Điều quan trọng là tất cả chúng đều **phân rã** (bởi vì tất cả các giá trị riêng đều dương), có nghĩa là gradient flow **luôn hội tụ** về trạng thái cân bằng mà tại đó hàm loss huấn luyện bằng 0.

### Thông qua chuỗi lập luận này, chúng ta đã chỉ ra rằng gradient descent hội tụ về 0 training loss cho bất kỳ mô hình phi tuyến tính nào miễn là nó gần với độ tuyến tính hóa của nó (có thể đạt được chỉ bằng cách lấy $\alpha \to \infty$)! Đây là điều cốt lõi của hầu hết các chứng minh trong các bài báo gần đây cho thấy rằng gradient descent đạt được 0 training loss.

# <b> <font color='black'> Quan sát Kernel Regime
---

### <font color=b> Phép toán gradient flow có thể hơi làm ta choáng ngợp, vì vậy chúng ta hãy cố gắng nắm bắt những gì đang xảy ra bằng cách quay lại ví dụ tập dữ liệu 2 điểm của chúng ta. Đối với 2 điểm dữ liệu, NTK chỉ là một ma trận xác định dương 2x2. Chúng ta có thể hình dung các ma trận như hình elip trong mặt phẳng 2-D, trong đó trục chính và trục phụ là các vector riêng (eigenvector), và độ dài của chúng tỷ lệ nghịch với căn bậc hai của giá trị riêng (eigenvalue). Hình elip về cơ bản là một đường bao của một hàm bậc hai biến thiên theo thời gian $(\boldsymbol y - \boldsymbol{\bar y})^T \boldsymbol H( \boldsymbol w(t)) (\boldsymbol y - \boldsymbol{\bar y})$.

### Trên hết, chúng ta có thể hình dung quỹ đạo của $\boldsymbol y( \boldsymbol w(t))$ được tạo ra bởi gradient descent (xấp xỉ gradient flow) trên mặt phẳng 2-D này. Quỹ đạo này tương đương với việc thực hiện gradient descent trên cùng một biến thiên theo thời gian hoặc *tức thời (instantaneous)*. Dưới đây là hình biểu diễn nó trông như thế nào:

<center>
<img src=images/ellipses107.gif width=500>
<img src=images/train_funcs108.gif width=500>

### Biểu diễn neural tangent kernel khi quá trình huấn luyện diễn ra đối với các quy mô mô hình khác nhau (bắt đầu từ các lần khởi tạo khác nhau). Ta đã chuẩn hóa gradient flow cho các $\alpha$ khác nhau sao cho tỷ lệ thời gian và tỷ lệ các tangent kernel khớp với nhau. Chúng ta có thể thấy rõ rằng $\alpha$ lớn hơn (màu hồng) dẫn đến NTK không đổi hơn, trong khi NTK cho $\alpha$ nhỏ hơn (màu đỏ) thay đổi đáng kể khi mô hình huấn luyện. Ở bên phải là mô hình hoạt động khi quá trình huấn luyện diễn ra.
</center>

- ### <font color=b>Ta cố tình căn giữa các hình elip trên dữ liệu mục tiêu $\boldsymbol{\bar y}$ để các quỹ đạo sẽ tiếp cận trung tâm. Bằng cách này, bạn có thể thấy rằng thành phần tương ứng với trục ngắn hơn (giá trị riêng lớn hơn) của hình elip hội tụ nhanh hơn trục dài hơn (giá trị riêng nhỏ hơn).
- ### Các giá trị nhỏ hơn của $\alpha$ cũng hội tụ về 0 sai số huấn luyện (đây được gọi là chế độ 'sâu' hoặc 'giàu'). Tuy nhiên, chúng ta không có bất kỳ dẫn chứng lý thuyết nào cho điều này. Chứng minh của chúng ta chỉ hoạt động khi $\alpha$ đủ lớn để hình elip không đổi trong suốt quá trình huấn luyện (chế độ hạt nhân - kernel regime).
- ### Các $\alpha$ nhỏ nhất hội tụ nhanh đến mức khó có thể nhận thấy trong ảnh gif.
- ### Ta đã sử dụng mạng hai lớp ẩn với độ rộng 100 cho những lần chạy này. Ta cũng đã cho đầu ra của các mạng này bằng 0 bằng cách trừ đi một bản sao khi khởi tạo (đó là lý do tại sao tất cả các quỹ đạo bắt đầu từ điểm gốc). Mặc dù chúng ta không thể hình dung được 10000 hoặc hơn trọng số của mạng này di chuyển như thế nào khi chúng ta huấn luyện, chúng ta vẫn có thể hình dung kết quả đầu ra của mạng và đó là những gì mà ảnh gif này cho thấy.

### Kết quả chính là: chế độ hạt nhân xảy ra như chúng ta mong đợi đối với alpha lớn, nhưng hành vi đối với alpha nhỏ không có một lời giải thích lý thuyết.

### **Một điểm bổ sung:**
### Chế độ NTK trong các đồ thị này xảy ra đối với $\alpha$ lớn. Chúng ta cũng có thể tiếp cận chế độ này bằng cách đưa chiều rộng đến vô cùng như chúng ta đã trình bày trước đó thay vì chỉ mở rộng đầu ra. Tuy nhiên, chúng ta nhận được thêm một điều đặc biệt trong trường hợp này. Lưu ý rằng các NTK khi khởi tạo là khác nhau, điều này là do việc khởi tạo là ngẫu nhiên. Nhưng, khi chúng ta đưa độ rộng ẩn đến vô cùng trong mạng nơ-ron, hóa ra hạt nhân lúc khởi tạo trở nên **xác định (deterministic)**! Có một NTK cố định đối với độ sâu và hàm kích hoạt nhất định. Trên thực tế, NTK này thực sự có thể được tính toán một cách chính xác, mô phỏng hiệu quả một mạng nơ-ron có độ rộng vô hạn (xem bài viết này [On Exact Computation in an Infinitely Wide Net](https://arxiv.org/abs/1904.11955)). Kết quả này rất giống với tiêu chuẩn gradient xác định mà chúng ta đã thấy trong bằng chứng mạng một lớp ẩn ở trên, nhưng ta sẽ không đi sâu vào chi tiết hơn, vì bài viết này đã đủ dài.

# <b> <font color='black'> Tính khái quát (Generalization)
---

### <font color=b> Cho đến nay, chúng ta chỉ nói về những gì xảy ra đối với dữ liệu huấn luyện. Nhưng còn bộ test, tức là các điểm ở giữa các mẫu huấn luyện của chúng ta thì sao?
### Thứ nhất, huấn luyện mô hình trong kernel regime tương đương với việc giải quyết một hệ thống tuyến tính. Tham số hóa quá mức $(p > n)$ chỉ có nghĩa là hệ thống tuyến tính này **chưa được xác định (under-determined)** - vì vậy nó có vô số nghiệm. Vì chúng ta đang sử dụng gradient descent để giải quyết hệ thống này, nên nó nó có những **bias tiềm ẩn (implicitly biased)** hướng tới giải pháp **minimum norm**. Nghĩa là, gradient descent chọn giải pháp có $\Vert \boldsymbol w \Vert_2$ tối thiểu (miễn là chúng ta bắt đầu từ lần khởi tạo với norm thấp).
### Nếu bây giờ chúng ta nghĩ đến việc mở rộng không gian $\boldsymbol y$ thành một **không gian hàm (function space)** có số chiều vô hạn, thì chuẩn hóa $\ell_2$ tối thiểu này trong không gian $w$ chuyển thành việc chọn một hàm làm tối thiểu một *hàm chuẩn hóa cụ thể*. Điều gì quyết định định norm này? Hạt nhân mô tả bài toán tuyến tính, trong trường hợp này là NTK. Điều này sau đó có thể được hiểu là một hình thức chính quy hóa và bạn có thể sử dụng điều này để nói về sự khái quát hóa. TA sẽ không đi sâu vào chi tiết hơn, vì bài đăng này chủ yếu nhằm mục đích tìm hiểu kết quả tối ưu hóa thay vì khái quát hóa. Nếu muốn tìm hiểu thêm về điều này, chúng ta có thể tìm kiếm thông tin về bài viết *reproducing kernel Hilbert spaces and the generalizing properties of kernel machines*.

# <b> <font color='black'> Kết luận
---

### <font color=b> Lý thuyết NTK rất tuyệt vời, nhưng những hình dung ở trên cho thấy rằng có nhiều thứ liên quan đến mạng nơ-ron hơn là chỉ chế độ hạt nhân này. Các thí nghiệm cho thấy rằng các mạng nơ-ron thành công trên thực tế chắc chắn **không hoạt động trong chế độ hạt nhân (do not operate in the kernel regime)**. Trên thực tế, ngay cả các mạng nơ-ron tuyến tính tốt nhất (các NTK được tính toán chính xác cho độ rộng vô hạn) cũng thực sự **hoạt động kém hơn (perform worse)** trên các điểm chuẩn tiêu chuẩn như MNIST và CIFAR khoảng ~ 7% (mặc dù những phát triển gần đây có thể đã thu hẹp khoảng cách này, hãy xem bài viết này [Enhanced Convolutional NTKs](https://arxiv.org/abs/1911.00809)).
### Tuy nhiên, lý thuyết chế độ NTK này không phải là lý thuyết duy nhất để hiểu rõ về training dynamics của neural network. Ngoài ra còn có một nguồn (xem tài liệu tham khảo) sử dụng các ý tưởng từ lý thuyết trường trung bình và vận chuyển tối ưu để mô tả đặc điểm động lực học khi tỷ lệ mô hình không lớn $(\kappa \sim 1)$ thông qua PDE phi tuyến tính, nhưng những lý thuyết này không mở rộng ra ngoài các mạng với một lớp ẩn duy nhất.
### Tuy nhiên, những kết quả của NTK này vẫn cực kỳ thú vị, bởi vì chúng cung cấp một cách nhìn mới về việc học mạng nơ-ron. Cố gắng hiểu hạt nhân thay đổi như thế nào khi quá trình huấn luyện tiến triển có vẻ như là một câu hỏi quan trọng cần trả lời trong quá trình tìm kiếm lý thuyết tốt hơn về mạng nơ-ron.
---
### Bạn có thể tìm thấy code cho bài viết này trong [repo này](https://github.com/rajatvd/NTK).